# Tratamiento de datos duplicados, ausentes o no válidos

## Acerca de los datos
En este notebook, utilizaremos datos meteorológicos diarios que fueron tomados de la [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) y alterados para introducir muchos problemas comunes a los que nos enfrentamos cuando trabajamos con datos.

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como se puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Antecedentes de los datos

Significado de los datos:
- `PRCP`: precipitación en milímetros
- `SNOW`: nevadas en milímetros
- `SNWD`: profundidad de la nieve en milímetros
- `TMAX`: temperatura máxima diaria en grados Celsius
- `TMIN`: temperatura mínima diaria en grados Celsius
- `TOBS`: temperatura en el momento de la observación en grados Celsius
- `WESF`: equivalente en agua de la nieve en milímetros

Algunos datos importantes para orientarnos:
- Según el Servicio Meteorológico Nacional, la temperatura más fría jamás registrada en Central Park fue de -26,1 °C (-15 °F) el 9 de febrero de 1934: [fuente](https://www.weather.gov/media/okx/Climate/CentralPark/extremes.pdf)
- La temperatura de la fotosfera del Sol es de aproximadamente 5.505°C: [fuente](https://en.wikipedia.org/wiki/Sun)

## Configuración
Necesitamos importar `pandas` y leer los datos sucios para empezar:

In [ ]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

## Encontrar datos problemáticos
Un buen primer paso es mirar algunas filas:

In [ ]:
df.head()

El examen de las estadísticas resumidas puede revelar valores extraños o ausentes:

In [ ]:
df.describe()

El método `info()` puede señalar valores omitidos y tipos de datos erróneos:

In [ ]:
df.info()

Podemos utilizar el método `isna()`/`isnull()` de la serie para encontrar nulos:

In [ ]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0]

In [ ]:
contain_nulls.head(10)

Tenga en cuenta que no podemos comprobar si tenemos `NaN` así:

In [ ]:
df[df.inclement_weather == 'NaN'].shape[0]

Esto se debe a que en realidad es `np.nan`. Sin embargo, observe que esto tampoco funciona:

In [ ]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

Tenemos que utilizar uno de los métodos comentados anteriormente para que esto funcione:

In [ ]:
df[df.inclement_weather.isna()].shape[0]

Podemos encontrar `-inf`/`inf` comparando con `-np.inf`/`np.inf`:

In [ ]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

En lugar de hacer esto para cada columna, podemos escribir una función que utilice un [diccionario de comprensión](https://www.python.org/dev/peps/pep-0274/) para comprobar todas las columnas por nosotros:

In [ ]:
def get_inf_count(df:pd.DataFrame) -> dict:
    """Encontrar el número de valores inf/inf por columna en el marco de datos"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

Antes de decidir cómo tratar los valores infinitos de la profundidad de nieve, debemos examinar las estadísticas de resumen de las nevadas, que constituyen una parte importante en la determinación de la profundidad de nieve:

In [ ]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

Veamos ahora las columnas `date` y `station`. Antes vimos el valor `?` de estación, así que sabemos que es el otro valor único. Sin embargo, vemos que algunas fechas están presentes 8 veces en los datos y sólo tenemos 324 días, lo que significa que también nos faltan días:

In [ ]:
df.describe(include='object')

Podemos utilizar el método `duplicated()` para encontrar filas duplicadas:

In [ ]:
df[df.duplicated()].shape[0]

El valor por defecto de `keep` es `'first'`, lo que significa que no mostrará la primera fila en la que se vieron los datos duplicados; sin embargo, podemos pasar `False` para verla:

In [ ]:
df[df.duplicated(keep=False)].shape[0]

También podemos especificar las columnas a utilizar:

In [ ]:
df[df.duplicated(['date', 'station'])].shape[0]

Veamos algunos duplicados. Sólo en los pocos valores que vemos aquí, sabemos que los 4 primeros están realmente en los datos 6 veces porque por defecto no estamos viendo su primera aparición:

In [ ]:
df[df.duplicated()].head()

## Cuestiones atenuantes

### Manejo de datos duplicados
Como sabemos que tenemos los datos meteorológicos de NY y nos hemos dado cuenta de que sólo tenemos dos entradas para `estación`, podemos decidir eliminar la columna `estación` porque sólo nos interesan los datos meteorológicos. Sin embargo, cuando tratamos con datos duplicados, tenemos que pensar en las ramificaciones de eliminarlos. Observe que sólo tenemos datos para la columna `WESF` cuando la estación es `?`:

In [ ]:
df[df.WESF.notna()].station.unique()

Si determinamos que no afectará a nuestro análisis, podemos utilizar `drop_duplicates()` para eliminarlos:

In [ ]:
# 1. hacer que la fecha sea una fecha-hora
df.date = pd.to_datetime(df.date)

# 2. guarde esta información para más tarde
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. ordenar al final
df.sort_values('station', ascending=False, inplace=True)

# 4. elimine los duplicados basándose en la columna de fecha manteniendo la primera ocurrencia
# que será la estación válida si tiene datos
df_deduped = df.drop_duplicates('date')

# 5. eliminar la columna de la estación porque hemos terminado con ella
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. tomar el WESF de la estación válida y volver a la estación ? si es nulo
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.shape

Aquí utilizamos el método `combine_first()` para unir los valores a la primera entrada no nula; esto significa que si tuviéramos datos de ambas estaciones, tomaríamos primero el valor proporcionado por la estación nombrada y si (y sólo si) esa estación fuera nula tomaríamos el valor de la estación nombrada `?`. En la tabla siguiente se muestran algunos ejemplos de esta situación:

| estación GHCND:USC00280907 | estación ? | resultado de `combine_first()` |
| :---: | :---: | :---: |
| 1 | 17 | 1 |
| 1 | `NaN` | 1 |
| `NaN` | 17 | 17 |
| `NaN` | `NaN` | `NaN` |

Fíjate en la 4ª fila&mdash;tenemos `WESF` en el lugar correcto gracias al índice:

In [ ]:
df_deduped.head()

### Tratamiento de los nulos
Podríamos eliminar los nulos, sustituirlos por algún valor arbitrario o imputarlos utilizando los datos circundantes. Cada una de estas opciones puede tener ramificaciones, por lo que debemos elegir sabiamente.

Podemos utilizar `dropna()` para eliminar filas en las que alguna columna tenga un valor nulo. Las opciones por defecto apenas nos dejan datos:

In [ ]:
df_deduped.dropna().shape

Si pasamos `how='all'`, podemos elegir que sólo se eliminen las filas en las que todo es nulo, pero esto no elimina nada:

In [ ]:
df_deduped.dropna(how='all').shape

Podemos utilizar sólo un subconjunto de columnas para determinar qué eliminar con el argumento `subset`:

In [ ]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

Esto también se puede realizar a lo largo de las columnas, y también podemos exigir un cierto número de valores nulos antes de eliminar los datos:

In [ ]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75).columns

Podemos elegir rellenar los valores nulos en su lugar con `fillna()`:

In [ ]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

En este punto hemos hecho todo lo posible sin distorsionar los datos. Sabemos que nos faltan fechas, pero si reindexamos, no sabemos cómo rellenar los datos `NaN`. Con los datos meteorológicos, no podemos suponer que porque nevó un día nevará el siguiente o que la temperatura será la misma. Por este motivo, tenga en cuenta que los siguientes ejemplos sólo tienen fines ilustrativos: que podamos hacer algo no significa que debamos hacerlo.

Dicho esto, vamos a tratar de resolver algunos de los problemas que quedan con los datos de temperatura. Sabemos que cuando `TMAX` es la temperatura del Sol, debe ser porque no había ningún valor medido, así que vamos a sustituirlo por `NaN`. También lo haremos para `TMIN` que actualmente utiliza -40°C como marcador de posición cuando sabemos que la temperatura más fría jamás registrada en NYC fue de -15°F (-26,1°C) el 9 de febrero de 1934:

In [ ]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

También supondremos que la temperatura no cambiará drásticamente día a día. Tenga en cuenta que esto es en realidad una gran suposición, pero nos permitirá entender cómo funciona `fillna()` cuando proporcionamos una estrategia a través del parámetro `method`. El método `fillna()` nos da 2 opciones para el parámetro `method`:
- `'ffill'` para rellenar hacia adelante
- bfill para rellenar hacia atrás

*Note that `'nearest'` is missing because we are not reindexing.*

Aquí usaremos `'ffill'` para mostrar cómo funciona:

In [ ]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.ffill(),
    TMIN=lambda x: x.TMIN.ffill()
).head()

Podemos utilizar `np.nan_to_num()` para convertir `np.nan` en 0 y `-np.inf`/`np.inf` en números finitos grandes negativos o positivos:

In [ ]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

Dependiendo de los datos con los que estemos trabajando, podemos utilizar el método `clip()` como alternativa a `np.nan_to_num()`. El método `clip()` permite limitar los valores a un umbral mínimo y/o máximo específico. Como `SNWD` no puede ser negativo, utilicemos `clip()` para imponer un límite inferior de cero. Para mostrar cómo funciona el límite superior, utilicemos el valor de `SNOW`:

In [ ]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

Podemos combinar `fillna()` con otros tipos de cálculos. Aquí sustituimos los valores perdidos de `TMAX` por la mediana de todos los valores `TMAX`, `TMIN` por la mediana de todos los valores `TMIN` y `TOBS` por la media de los valores `TMAX` y `TMIN`. Como colocamos `TOBS` en último lugar, tenemos acceso a los valores imputados para `TMIN` y `TMAX` en el cálculo:

In [ ]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

También podemos utilizar `apply()` para ejecutar el mismo cálculo en todas las columnas. Por ejemplo, vamos a rellenar todos los valores que faltan con la mediana móvil de 7 días de sus valores, estableciendo el número de periodos necesarios para el cálculo en 0 para asegurarnos de que no introducimos más valores `NaN` adicionales. Los cálculos continuos se tratarán en el capítulo 4, así que esto es un avance:

In [ ]:
df_deduped.apply(
    # los cálculos continuos se tratarán en el capítulo 4, esta es una mediana continua de 7 días
    # fijamos min_periods (# de periodos requeridos para el cálculo) a 0 para que siempre obtengamos un resultado
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

La última estrategia que podemos probar es la interpolación con el método `interpolate()`. Especificamos el parámetro `method` con la estrategia de interpolación a utilizar. Hay muchas opciones, pero nos quedaremos con la predeterminada `'linear'`, que tratará los valores como espaciados uniformemente y colocará los valores perdidos en medio de los existentes. Tenemos algunos datos que faltan, así que primero reindexaremos. Los valores de `TMAX`, `TMIN` y `TOBS` son la media de los valores del día anterior (8 de enero) y del día posterior (10 de enero):

In [ ]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="width: 33.33%; text-align: left;">
        <a href="./4-reshaping_data.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="width: 33.33%; text-align: center;">
        <a href="../solutions/ch_03/solutions.ipynb">
            <button>Soluciones</button>
        </a>
    </div>
    <div style="width: 33.33%; text-align: right;">
        <a href="../ch_04/1-consulta_y_merge.ipynb">
            <button>Capitulo 4 &#8594;</button>
        </a>
    </div>
</div>

<hr>
